In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# -------------------------------
# Step 1: 读取数据
# -------------------------------
df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据_2.csv',
                 sep=',', engine='python', encoding='utf-8')

# 去除制表符
df = df.map(lambda x: x.strip('\t') if isinstance(x, str) else x)

# -------------------------------
# Step 2: 定义原始特征列（sx_cols）
# -------------------------------
sx_cols = [
    '课前预学','课堂参与','课后复习','延伸阅读',
    '完成作业时间','自习时间','课外阅读时间','网络课程时间',
    '实验科研时间','社团活动时间','竞赛活动时间','其他学习时间',
    '同学合作','参与科研团队','参与学科竞赛','学习同学方法','师生交流频度'
]

# 转为 float
df[sx_cols] = df[sx_cols].astype(float)

In [ ]:
# -------------------------------
# Step 3: 数据标准化
# -------------------------------
X = df[sx_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# -------------------------------
# Step 4: PCA 降维
# -------------------------------
# 保留前 6 个主成分（可调整 n_components）
pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print("PCA 解释的累计方差比例:", np.sum(pca.explained_variance_ratio_))

In [ ]:
# 输出每个主成分解释的方差比例
explained_ratios = pca.explained_variance_ratio_
cum_var = np.cumsum(explained_ratios)

print("PCA 各主成分方差贡献率：")
for i, ratio in enumerate(explained_ratios, start=1):
    print(f"主成分 {i}: {ratio:.3f}, 累计: {cum_var[i-1]:.3f}")


#可视化方差贡献率

plt.figure(figsize=(7,5))
plt.bar(range(1, len(explained_ratios)+1), explained_ratios, alpha=0.7, align='center', label='单个主成分方差贡献率')
plt.step(range(1, len(cum_var)+1), cum_var, where='mid', label='累计方差贡献率', color='red')

plt.xlabel('主成分序号')
plt.ylabel('方差解释比例')
plt.title('PCA 主成分方差贡献率')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
# 获取每个主成分的特征向量（载荷）
loadings = pca.components_

# 转换成 DataFrame，更直观
loading_df = pd.DataFrame(loadings.T,
                          columns=[f'PC{i+1}' for i in range(loadings.shape[0])],
                          index=['完成作业时间','自习时间','课外阅读时间','网络课程时间','其他学习时间',
                                 '课前预学','课堂参与','课后复习','延伸阅读','学习同学方法',
                                 '实验科研时间','参与科研团队','参与学科竞赛','竞赛活动时间',
                                 '同学合作','社团活动时间','师生交流频度'])

print("PCA 载荷矩阵：")
print(loading_df.round(3))


In [ ]:
# -------------------------------
# Step 5: 确定聚类数 K (肘部法 + 轮廓系数)
# -------------------------------
wcss = []
silhouette_scores = []

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_pca, kmeans.labels_))

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(range(2,10), wcss, 'o-', label='WCSS')
plt.xlabel('聚类数 K')
plt.ylabel('WCSS')
plt.title('肘部法')

plt.subplot(1,2,2)
plt.plot(range(2,10), silhouette_scores, 'o-', label='轮廓系数')
plt.xlabel('聚类数 K')
plt.ylabel('轮廓系数')
plt.title('轮廓系数')
plt.show()


In [ ]:
# -------------------------------
# Step 6: 聚类建模 (KMeans, 假设 4 类)
# -------------------------------
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X_pca)

# -------------------------------
# Step 7: 映射画像标签 (可人工命名)
# -------------------------------
cluster_mapping = {
    0: '科研学霸型',
    1: '社团活跃型',
    2: '学业挣扎型',
    3: '自主学习型'
}
df['student_persona'] = df['cluster'].map(cluster_mapping)

# -------------------------------
# Step 8: 统计分析
# -------------------------------
print("各画像人数分布：")
print(df['student_persona'].value_counts())

persona_means = df.groupby('student_persona')[sx_cols].mean()
print("\n各画像特征均值：")
print(persona_means)

# -------------------------------
# Step 9: 可视化
# -------------------------------
# (1) 各画像人数柱状图
sns.countplot(x='student_persona', data=df,
              order=df['student_persona'].value_counts().index)
plt.title("各画像人数分布")
plt.show()

In [ ]:
# (2) PCA 前两维度可视化聚类效果
plt.figure(figsize=(6,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=df['student_persona'], palette="Set2")
plt.title("PCA降维后的聚类分布（前2主成分）")
plt.xlabel("主成分1")
plt.ylabel("主成分2")
plt.legend()
plt.show() 

In [ ]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 创建标签编码器
le = LabelEncoder()
# 将字符串标签转换为数值
numeric_labels = le.fit_transform(df['student_persona'])

# (2) PCA 三维聚类结果可视化
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制三维散点图 - 使用数值标签
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                     c=numeric_labels, cmap="Set2", s=50, alpha=0.8)

# 设置标题和坐标轴标签
ax.set_title("PCA降维 - 聚类分布（前三主成分）", fontsize=14)
ax.set_xlabel("主成分1", fontsize=12)
ax.set_ylabel("主成分2", fontsize=12)
ax.set_zlabel("主成分3", fontsize=12)

# 添加图例 - 需要手动创建图例
legend_labels = le.classes_
handles = [plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=plt.cm.Set2(i/len(legend_labels)),
                      markersize=10) for i in range(len(legend_labels))]
ax.legend(handles, legend_labels, title="学生画像", loc="upper right", fontsize=10)

# 显示图形
plt.tight_layout()
plt.show()